In [ ]:
# measure weights for mulitple epochs
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import train
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-s', '--seed',
    type=float,
    default=1,
    help="Seed for randomness",
    dest="seed"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer
tf.random.set_seed(args.seed)

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
model,weights=train.weight_measure(model,layer_str,10,final_learning_rate,x_train,y_train,batch_size) # for 5000 epochs use "measure_epochs_only=True"
np.save(model_str+'/weights_'+layer_str,weights)

In [ ]:
# compute the principal components
import numpy as np
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
args=parser.parse_args()
model_str=args.name
layer_str=args.layer

weights=np.load(model_str+'/weights_'+layer_str+'.npy')
Cov=np.cov(weights.T)
variance,pcomp_cov=np.linalg.eigh(Cov)
pcomp=pcomp_cov.T[::-1]
np.save(model_str+'/pc_'+layer_str,pcomp)

In [ ]:
# compute theta
import numpy as np
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
args=parser.parse_args()
model_str=args.name
layer_str=args.layer

weights=np.load(model_str+'/weights_'+layer_str+'.npy')
pcomp=np.load(model_str+'/pc_'+layer_str+'.npy')
theta=np.tensordot(weights,pcomp,axes=[1,1])
np.save(model_str+'/theta_'+layer_str,theta.T)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import analysis
#plt.rcParams['text.usetex'] = False

import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, all or layer of sv",
    dest="layer"
)
parser.add_argument(
    '-m', '--mode',
    type=str,
    default="evh",
    help="type of input vectors, evh or pc",
    dest="mode"
)

parser.add_argument(
    '-lx', '--lyrdex',
    type=int,
    default="-1",
    help="layer index for singular values",
    dest="lx"
)

args = parser.parse_args()
model_str = args.name
md_str = args.mode
layer_index = args.lx
layer_str = args.layer


exec(open(model_str+'/'+model_str+".py").read())
field = analysis.sv_field(model, model_str, layer_str, md_str='pc')
if md_str == "evh":
    if layer_str == "all":
        np.save(model_str+'/field_'+str(layer_index), field)
    else:
        np.save(model_str+'/field_'+layer_str, field)
else:
    np.save(model_str+'/field_'+layer_str+'_'+md_str, field)


In [ ]:
# compute accuracies and losses for different number of components added
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import analysis
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)
parser.add_argument(
    '-m', '--mode',
    type=str,
    default="evh",
    help="type of input vectors, evh or pc",
    dest="mode"
)
parser.add_argument(
    '-a', '--abs',
    type=bool,
    default=True,
    help="whether eigenvalues should be added in order of magnitude or sign",
    dest="abs"
)

args=parser.parse_args()
model_str=args.name
md_str=args.mode
layer_str=args.layer


exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
if md_str=="evh":
        if args.abs:
            sort_list=np.argsort(np.abs(np.load(model_str+'/ewh_'+layer_str+'.npy')))
            print(sort_list)
            print(np.take(np.load(model_str+'/ewh_'+layer_str+'.npy'),sort_list,axis=0)[::-1])
            vec=np.take(np.load(model_str+'/evh_'+layer_str+'.npy'),sort_list,axis=0)[::-1]
        else:
            vec=np.load(model_str+'/evh_'+layer_str+'.npy')
else:
    vec=np.load(model_str+'/'+md_str+'.npy')
accs=analysis.acc_components(model,layer_str,vec,x_train,y_train,x_test,y_test)
if args.abs:
    np.save(model_str+'/accabs_'+layer_str+md_str,accs)
else:
    np.save(model_str+'/acc_'+layer_str+md_str,accs)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import time
#plt.rcParams['text.usetex'] = False



import argparse
parser = argparse.ArgumentParser(
    prog='train MLP256',
    description='train MLP256',
    epilog='Based on the rmt package.')
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-r', '--reg',
    type=float,
    default=0,
    help="Regularization",
    dest="reg"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer
if args.reg!=0:
  reg=tf.keras.regularizers.l2(args.reg)
else:
  reg=None


exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
layer_name=[]
b_list=[]
if layer_str=="all":
  for layer in model.trainable_weights:
    if len(layer.shape)==1:
      b_list.append(np.array(layer))
    if len(layer.shape)>0:
      layer_name.append(np.array(layer))
else:
  for layer in getattr(model,layer_str).get_weights():
    if len(layer.shape)==1:
      b_list.append(layer)
    if len(layer.shape)>1:
      layer_name.append(layer)
print(len(layer_name))
print(len(b_list))
#computes the number of parameters of the layer
layer_size = 0
for layer in layer_name:
    layer_size += np.prod(layer.shape)
#conserve the numpy datatype
layer_dtype = np.float32
layer_lengths=[]
for layer in layer_name:
  layer_lengths.append(tf.math.reduce_prod(layer.shape))
batch_size=12500
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(batch_size)
"""if layer_str=="all":
  evh=np.load(model_str+'/ev_sparse'+'.npy').T[::-1]"""

#evh=np.load(model_str+'/evh_'+layer_str+'.npy')
#ewh=np.load(model_str+'/ewh_'+layer_str+'.npy')

@tf.function
def flatten(grad):
    temp = tf.TensorArray(tf.float32, size=0, dynamic_size=True, infer_shape=False)
    for gr in grad:
        temp = temp.write(temp.size(), tf.reshape(gr, (tf.math.reduce_prod(tf.shape(gr)), )))
    return temp.concat()
weights_0=flatten(layer_name)

@tf.function
def repack(weights):
    weight_split=tf.split(weights,layer_lengths)
    templer = []
    for layer,weights_set in zip(layer_name,weight_split):
        templ = tf.reshape(weights_set,layer.shape)
        templer.append(templ)
    return templer

if layer_str=="all":
  def set_weights(weights):
    templ=repack(weights)
    """for i in range(len(layer_name)):
      model.layers[i+1].set_weights([templ[i],b_list[i]])"""
    k=0
    for i in range(len(model.layers)):
        if len(model.layers[i].get_weights())==2:
            model.layers[i].set_weights([templ[k],templ[k+1]])
            k+=2
        elif len(model.layers[i].get_weights())==1:
            model.layers[i].set_weights([templ[k]])
            k+=1
else:
  def set_weights(weights):
    templ=repack(weights)
    getattr(model,layer_str).set_weights([templ[0],b_list[0]])


test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
train_loss = tf.keras.metrics.Mean(name='train_loss')

@tf.function
def calc_acc(images, labels):
    predictions = model(images, training=False)
    test_accuracy(labels, predictions)
    loss=loss_object(labels, predictions)
    loss+=sum(model.losses)
    train_loss(loss)
#weights_prod=np.tensordot(weights_0,evh,axes=(0,1))
#max_entry=np.argmax(weights_prod)
step_range=0#evh.shape[0]

step_array=np.concatenate(([0],[1],[2]))
shift_step=0.01
#for i in step_array:
#shift=weights_prod[max_entry]
epsilons=np.arange(-2,4,shift_step)
trainacc=np.empty(epsilons.shape[0])
trainloss=np.empty(epsilons.shape[0])
testacc=np.empty(epsilons.shape[0])
index=0
for images, labels in train_ds:
      calc_acc(images, labels)
loss_0=train_loss.result()
train_loss.reset_states()
test_accuracy.reset_states()
incl_vecs=np.arange(-step_range,step_range+1,1)
print(incl_vecs)
for epsilon in epsilons:
    weights_set=weights_0#-np.tensordot(evh[incl_vecs],weights_prod[incl_vecs],axes=[0,0])
    
    for i in incl_vecs:
        weights_set+=(epsilon)*weights_0
    set_weights(weights_set)
    for images, labels in train_ds:
      calc_acc(images, labels)
    trainacc[index]=test_accuracy.result()
    trainloss[index]=train_loss.result()
    train_loss.reset_states()
    test_accuracy.reset_states()
    calc_acc(x_test,y_test)
    testacc[index]=test_accuracy.result()
    train_loss.reset_states()
    test_accuracy.reset_states()
    index+=1
np.save(model_str+'/loss_scaling',[epsilons,trainacc,trainloss,testacc])

In [ ]:
# compute loss landscape
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import argparse
parser = argparse.ArgumentParser(
    prog='train MLP256',
    description='train MLP256',
    epilog='Based on the rmt package.')
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer
if args.reg!=0:
  reg=tf.keras.regularizers.l2(args.reg)
else:
  reg=None


exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
layer_name=[]
b_list=[]
if layer_str=="all":
  for layer in model.trainable_weights:
    if len(layer.shape)==1:
      b_list.append(np.array(layer))
    if len(layer.shape)>0:
      layer_name.append(np.array(layer))
else:
  for layer in getattr(model,layer_str).get_weights():
    if len(layer.shape)==1:
      b_list.append(layer)
    if len(layer.shape)>1:
      layer_name.append(layer)
print(len(layer_name))
print(len(b_list))
#computes the number of parameters of the layer
layer_size = 0
for layer in layer_name:
    layer_size += np.prod(layer.shape)
#conserve the numpy datatype
layer_dtype = np.float32
layer_lengths=[]
for layer in layer_name:
  layer_lengths.append(tf.math.reduce_prod(layer.shape))
batch_size=12500
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(batch_size)
"""if layer_str=="all":
  evh=np.load(model_str+'/ev_sparse'+'.npy').T[::-1]"""

evh=np.load(model_str+'/evh_'+layer_str+'.npy')
ewh=np.load(model_str+'/ewh_'+layer_str+'.npy')

@tf.function
def flatten(grad):
    temp = tf.TensorArray(tf.float32, size=0, dynamic_size=True, infer_shape=False)
    for gr in grad:
        temp = temp.write(temp.size(), tf.reshape(gr, (tf.math.reduce_prod(tf.shape(gr)), )))
    return temp.concat()
weights_0=flatten(layer_name)

@tf.function
def repack(weights):
    weight_split=tf.split(weights,layer_lengths)
    templer = []
    for layer,weights_set in zip(layer_name,weight_split):
        templ = tf.reshape(weights_set,layer.shape)
        templer.append(templ)
    return templer

if layer_str=="all":
  def set_weights(weights):
    templ=repack(weights)
    """for i in range(len(layer_name)):
      model.layers[i+1].set_weights([templ[i],b_list[i]])"""
    k=0
    for i in range(len(model.layers)):
        if len(model.layers[i].get_weights())==2:
            model.layers[i].set_weights([templ[k],templ[k+1]])
            k+=2
        elif len(model.layers[i].get_weights())==1:
            model.layers[i].set_weights([templ[k]])
            k+=1
else:
  def set_weights(weights):
    templ=repack(weights)
    getattr(model,layer_str).set_weights([templ[0],b_list[0]])


test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
train_loss = tf.keras.metrics.Mean(name='train_loss')

@tf.function
def calc_acc(images, labels):
    predictions = model(images, training=False)
    test_accuracy(labels, predictions)
    loss=loss_object(labels, predictions)
    loss+=sum(model.losses)
    train_loss(loss)
weights_prod=np.tensordot(weights_0,evh,axes=(0,1))
max_entry=np.argmax(weights_prod)
step_range=0#evh.shape[0]

step_array=np.concatenate(([0],[max_entry]))
print(step_array)
shift_step=0.01
for i in step_array:
    shift=weights_prod[i]
    epsilons=np.arange(shift-weights_prod[max_entry]-shift_step,2*shift_step+shift+weights_prod[max_entry],shift_step)
    trainacc=np.empty(epsilons.shape[0])
    trainloss=np.empty(epsilons.shape[0])
    testacc=np.empty(epsilons.shape[0])
    index=0
    for images, labels in train_ds:
          calc_acc(images, labels)
    loss_0=train_loss.result()
    train_loss.reset_states()
    test_accuracy.reset_states()
    incl_vecs=np.arange(-step_range+i,step_range+i+1,1)
    print(incl_vecs)
    for epsilon in epsilons:
        weights_set=weights_0#-np.tensordot(evh[incl_vecs],weights_prod[incl_vecs],axes=[0,0])
        
        for i in incl_vecs:
            weights_set+=(epsilon-weights_prod[i])*evh[i]
        set_weights(weights_set)
        for images, labels in train_ds:
          calc_acc(images, labels)
        trainacc[index]=test_accuracy.result()
        trainloss[index]=train_loss.result()
        train_loss.reset_states()
        test_accuracy.reset_states()
        calc_acc(x_test,y_test)
        testacc[index]=test_accuracy.result()
        train_loss.reset_states()
        test_accuracy.reset_states()
        index+=1
    np.save(model_str+'/eps/evh_eps_'+str(i),[epsilons,trainacc,trainloss,testacc])